In [1]:
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import sklearn
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from polyfuzz import PolyFuzz
import Levenshtein

nltk.download("stopwords")
from nltk.corpus import stopwords
stopwords=stopwords.words("english")

ModuleNotFoundError: No module named 'polyfuzz'

In [10]:
!pip install python-Levenshtein
!pip install polyfuzz[all]

     |████████████████████████████████| 757kB 12.6MB/s 
     |████████████████████████████████| 14.5MB 286kB/s 
     |████████████████████████████████| 112kB 53.1MB/s 
     |████████████████████████████████| 286kB 47.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 2.2MB 37.1MB/s 
     |████████████████████████████████| 1.2MB 39.5MB/s 
     |████████████████████████████████| 983kB 43.8MB/s 
     |████████████████████████████████| 798kB 46.8MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 870kB 32.1MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=23255433f24146240194f15aa332b31218d3ce640a6106befa3b321ed9b42b77
 

In [24]:
sentences=[
           "This is a foobar sentence",
           "This sentence is not similar to a foobar sentence",
           "This is another string, but it is not quite similar to the previous ones",
           "I am also just another string",
           "Again not a foobar sentence"
]

In [3]:
def clean_string(text):
  text = ''.join([word for word in text if word not in string.punctuation])
  text = text.lower()
  text = ' '.join([word for word in text.split() if word not in stopwords])
  return text

In [4]:
cleaned = list(map(clean_string,sentences))
print(cleaned)

['foobar sentence', 'sentence similar foobar sentence', 'another string quite similar previous ones', 'also another string']


In [5]:
vectorizer = CountVectorizer().fit_transform(cleaned)
vectors = vectorizer.toarray()
print(vectors)

[[0 0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 2 1 0]
 [0 1 0 1 1 1 0 1 1]
 [1 1 0 0 0 0 0 0 1]]


In [6]:
csim = cosine_similarity(vectors)
print(csim)

[[1.         0.8660254  0.         0.        ]
 [0.8660254  1.         0.16666667 0.        ]
 [0.         0.16666667 1.         0.47140452]
 [0.         0.         0.47140452 1.        ]]


In [7]:
Levenshtein.distance(sentences[2],sentences[1])

46

In [16]:
from_list = ["this is an apple", "these are apples"]
to_list = ["these are many apples"]

model = PolyFuzz("EditDistance")
# model = PolyFuzz("TF-IDF")
# model = PolyFuzz("Embeddings")

model.match(from_list, to_list)

model.get_matches()

2021-04-11 16:55:05,322 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpgjasun0s


100%|██████████| 1200000128/1200000128 [01:06<00:00, 18018851.18B/s]

2021-04-11 16:56:12,478 copying /tmp/tmpgjasun0s to cache at /root/.flair/embeddings/en-fasttext-news-300d-1M.vectors.npy


2021-04-11 16:56:23,979 removing temp file /tmp/tmpgjasun0s
2021-04-11 16:56:24,619 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M not found in cache, downloading to /tmp/tmp4hjvzhsd


100%|██████████| 54600983/54600983 [00:03<00:00, 14079988.28B/s]

2021-04-11 16:56:29,040 copying /tmp/tmp4hjvzhsd to cache at /root/.flair/embeddings/en-fasttext-news-300d-1M
2021-04-11 16:56:29,110 removing temp file /tmp/tmp4hjvzhsd


,From,To,Similarity
0,this is an apple,these are many apples,0.742809
1,these are apples,these are many apples,0.976916


In [25]:
tfidf = TfidfVectorizer().fit_transform(sentences)
pairwise_similarity = tfidf * tfidf.T
arr=pairwise_similarity.toarray()
np.fill_diagonal(arr, np.nan)
result_idx = np.nanargmax(arr[tfidf.shape[0]-1])
print(sentences[result_idx])

This sentence is not similar to a foobar sentence
